In [8]:
import pandas as pd

# Load your Excel file
df = pd.read_excel("ThreeFilesMerged.xlsx")
#print(df)

# Make sure we only use the required columns
required_columns = {'source', 'page_number', 'chunk_number', 'text'}
df = df[list(required_columns)]

In [12]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from tqdm import tqdm

# Load Excel data
df = pd.read_excel("ThreeFilesMerged.xlsx")

# Load the model with trust_remote_code
model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

# Extract the texts
texts = df["text"].tolist()

# Generate embeddings
embeddings = model.encode(texts, batch_size=64, show_progress_bar=True)

# You can now use the embeddings to build a FAISS index or save them


!!!!!!!!!!!!megablocks not available, using torch.matmul instead
<All keys matched successfully>
Batches: 100%|███████████████████████████████████████████████████████████████████████████| 5/5 [03:43<00:00, 44.65s/it]


In [13]:
import faiss
import numpy as np
import pickle
import os

# Ensure embeddings are in float32 (FAISS requires this)
embedding_array = np.array(embeddings).astype('float32')

# Create FAISS index (flat L2 distance; can use other types if needed)
index = faiss.IndexFlatL2(embedding_array.shape[1])

# Add vectors
index.add(embedding_array)

# Store metadata: map FAISS index to your original DataFrame rows
metadata = df[['source', 'page_number', 'chunk_number', 'text']].to_dict(orient='records')

# Save FAISS index
faiss.write_index(index, "vector_index.faiss")

# Save metadata (as a list of dicts)
with open("vector_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)

print("✅ FAISS index and metadata saved!")



✅ FAISS index and metadata saved!


In [ ]:
#Retrieval-Augmented Generation (RAG) Q&A system

In [14]:
import faiss
import pickle
import numpy as np

# Load FAISS index and metadata
index = faiss.read_index("vector_index.faiss")

with open("vector_metadata.pkl", "rb") as f:
    metadata = pickle.load(f)


In [15]:
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)

def embed_query(query):
    return embed_model.encode([query])[0].astype('float32')


<All keys matched successfully>


In [16]:
def search_index(query, k=5):
    query_embedding = embed_query(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, k)
    return [metadata[i] for i in indices[0]]


In [20]:
from llama_cpp import Llama

llm = Llama(
    model_path="C:/Users/salsubhi1/models/llama-2-7b-chat.Q4_K_M.gguf",
    n_ctx=8192,
    n_threads=4
)


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from C:/Users/salsubhi1/models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head

llama_kv_cache_init: layer 17: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 18: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 19: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 20: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 21: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 22: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 23: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 24: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 25: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 26: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 27: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 28: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 29: n_embd_k_gqa = 4096, n_embd_v_gqa = 4096
llama_kv_cache_init: layer 30: n_embd_k_gqa = 4096, n_embd_v_gqa

In [21]:
def rag_answer(question, top_k=5):
    # 1. Retrieve top-k chunks
    retrieved_chunks = search_index(question, k=top_k)
    context = "\n\n".join([chunk["text"] for chunk in retrieved_chunks])

    # 2. Prompt LLaMA with retrieved context + question
    prompt = f"""You are a helpful assistant. Use the following context from publications to answer the question.\n\nContext:\n{context}\n\nQuestion: {question}\nAnswer:"""

    response = llm(prompt, max_tokens=300, stop=["\n\n", "Question:"], echo=False)
    return response["choices"][0]["text"].strip()


In [22]:
question = "What are the main study of the researcher?"
answer = rag_answer(question)
print("Answer:", answer)


llama_perf_context_print:        load time =  152970.05 ms
llama_perf_context_print: prompt eval time =  152969.27 ms /  3500 tokens (   43.71 ms per token,    22.88 tokens per second)
llama_perf_context_print:        eval time =    5894.49 ms /    26 runs   (  226.71 ms per token,     4.41 tokens per second)
llama_perf_context_print:       total time =  158870.62 ms /  3526 tokens


Answer: The main study of the researcher is to explore the effects of old age on the mental health of the aged people.
